# Game of life

This is an interacive jupyter notebook that lets you edit the code in any of the cells. All the code needs to be in Python 3. 
To run a part of the code, select a cell by clicking on it and press Run button from the toolbar. To run all cells in the notebook at once, select the Cell option from the toolbar and click Run all. You may use any of the imported libraries in the first cell. 

Your task is to fill in the check_neighbours function that takes the current generation array and updates it according to the rules of Conway's Game of Life. The function is passed an array with the cells and an integer N that is size of the array, however your are not required to use it. The function needs to return the updated array after one time step.
### Some helpful functions:
* array = np.zeros((n,m)) - will create an n x m array filled with zeroes 
* new_array = array_name.copy() - will copy the selected array so you can edit it without making changes in the original array
* n % m - n is divided by m, and the remainder is returned


### Arrays:
The array indexes start at 0 and end at size-1. That means if you have 20x30 array accessing cell at array[19][29] will be okay but array[20][30] will cause an error. The simulation uses a wrap around array, meaning the first row is connected to the last and the same goes for columns. This is a useful tool as it prevents out-of-bounds errors. The simulation array is filled with 1s and 0s, 1s representing the live cells, 0s representing the empty spaces.

To get started, run all of the code cells. This should show the grid with buttons, but the core algorithm is missing.

In [1]:
#The first set of cells is for setting up the grid only, you do not need to edit these.
import numpy as np
import time
from ipycanvas import hold_canvas, MultiCanvas
import ipywidgets as widgets
from ipywidgets import Button, HBox, VBox, Layout
import threading
import random


In [2]:
#set up canvas
def draw_grid(canvas):
    canvas.stroke_style = '#c1bcbb'
    for x in range(1,500,10):
        canvas.begin_path()
        canvas.move_to(x, 0)
        canvas.line_to(x, 500)
        canvas.stroke()
        
    canvas.move_to(500, 0)
    canvas.line_to(500, 500)
    canvas.stroke() 
    
        
    for x in range(1,500,10):
        canvas.begin_path()
        canvas.move_to(0, x)
        canvas.line_to(500, x)
        canvas.stroke() 
        
    canvas.move_to(0, 500)
    canvas.line_to(500, 500)
    canvas.stroke()
        


In [3]:
#draw the current state of the simulation array
def draw_cells(live_cells, canvas, cell_size):
    with hold_canvas(canvas):
        canvas.clear()
        r = 0
        for row in live_cells:
            c = 0
            for value in row:
                if value:
                    canvas.fill_rect(r * cell_size, c * cell_size, cell_size)

                c += 1
            r += 1


In [4]:
#randomly fills the array will live cells
def random_setup(live_cells):
    for i in range(50):
        for j in range(50):
            live_cells[i][j] = random.randint(0,1)
            
    

In [5]:
#some special cases for the simulation
def on_change(selected):
    global canvas
    global live_cells
    
    live_cells = np.zeros((50, 50))
    if selected['new']==1 :
        pulsar =\
        [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0],
         [0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0],
         [0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0],
         [0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0],
         [0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0],
         [0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0],
         [0,0,1,0,0,0,0,1,0,1,0,0,0,0,1,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,1,1,1,0,0,0,1,1,1,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],]
        
        live_cells[1:18,1:18] = pulsar
            
    if selected['new']==2 :
        beacon =\
        [[0,0,0,0,0,0], 
         [0,1,1,0,0,0],
         [0,1,1,0,0,0],
         [0,0,0,1,1,0],
         [0,0,0,1,1,0],
         [0,0,0,0,0,0]]
        
        live_cells[1:7,1:7] = beacon
        
    if selected['new']==3 :
        spaceship=\
        [[0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,1,1,0,0,0,0],
         [0,0,0,1,0,0,0,0,1,0,0],
         [0,0,1,0,0,0,0,0,0,0,0],
         [0,0,1,0,0,0,0,0,1,0,0],
         [0,0,1,1,1,1,1,1,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0]]
        
        live_cells[1:10,1:12] = spaceship
        
    if selected['new']==4 :
        penta=\
        [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0],
         [0,0,0,0,1,1,0,1,1,1,1,0,1,1,0,0,0,0],
         [0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
         [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
        
        live_cells[1:12,1:19] = penta
        
    if selected['new']==5 :
            glider_gun =\
            [[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1],
             [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,1],
             [1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
             [1,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],
             [0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]]
            
            live_cells[1:10,1:37] = glider_gun
    
    if selected['new']==6 :
        random_setup(live_cells)
        
    draw_cells(live_cells, canvas[1], 10)  

In [6]:
#simulation function, will run until stop or pause button is pressed
def play_game():
    global play
    play = True
    global live_cells
    global canvas
    draw_cells(live_cells, canvas[1], 10)
    while play:
        live_cells = check_neighbours(live_cells, 50)
        draw_cells(live_cells, canvas[1], 10)
        time.sleep(0.1)

In [7]:
#start the simulation button
def start_game(b):
    thread = threading.Thread(target=play_game)
    thread.start()

In [8]:
# clear canvas button
def clear_game(b):
    global live_cells
    global canvas
    global play
    play = False
    canvas[1].clear()
    live_cells = np.zeros((50, 50))

In [9]:
#pause button
def pause_game(b):
    global play 
    play = False

In [10]:
# TASK 
# receives current state of the simulation as n x n array and integer size of the array
# returns next generation array for the given input
def check_neighbours (live_cells, N):

 
    return(live_cells)

In [11]:
#widget to display the simulation and the buttons
live_cells = np.zeros((50, 50))
play = True

start_button = Button(description="start")
stop_button = Button(description="stop")
clear_button = Button(description="clear")
select_button = widgets.Dropdown(
    options=[('Select', 0), ('Pulsar', 1), ('Beacon', 2), ('Spaceship', 3), ('Penta-decathlon', 4),('Glider gun', 5),('Random', 6)],
    value = 0,
    layout={'width': 'max-content'}
)

canvas = MultiCanvas(n_canvases=3, width=500, height=500)
draw_grid(canvas[0])
interaction_layer = canvas[2]
right_box = VBox([start_button, stop_button, clear_button, widgets.Label(value =' '), widgets.Label(value ='Patterns: '), select_button])
HBox([canvas,right_box])



In [12]:
start_button.on_click(start_game)
stop_button.on_click(pause_game)
clear_button.on_click(clear_game)
select_button.observe(on_change, names='value')

There are 3 test cases to check if your function is working properly. A random set up will be passed to your function and the output will be compared to the expected one. To complete the exercise, pass all 3 test cases with your function.

In [13]:
cells_to_test = np.loadtxt("tests/testcase1.txt")
result = np.loadtxt("tests/testsolve1.txt")
if (np.array_equal(check_neighbours(cells_to_test,50), result)):
    print("Passed testcase 1")
else:
    print("Failed testcase 1")

Failed testcase 1


In [14]:
cells_to_test = np.loadtxt("tests/testcase2.txt")
result = np.loadtxt("tests/testsolve2.txt")
if (np.array_equal(check_neighbours(cells_to_test,50), result)):
    print("Passed testcase 2")
else:
    print("Failed testcase 2")
    

Failed testcase 2


In [15]:
cells_to_test = np.loadtxt("tests/testcase3.txt")
result = np.loadtxt("tests/testsolve3.txt")
if (np.array_equal(check_neighbours(cells_to_test,50), result)):
    print("Passed testcase 3")
else:
    print("Failed testcase 3")

Failed testcase 3
